<h1>Annual Change in Debt Redemption</h1>
Create a bar graph showing the change in the amount of debt redeemed from year to year and month to month 

In [3]:
using CSV, DataFrames, DataFramesMeta, Gadfly, XLSX, Dates, Compose, Plots

In [4]:
function clean_df(df) 
    df[:, :dollars] = replace(df[:, :dollars], "NaN" => missing) # replace "NaN" with missing
    df[:, :cents] = replace(df[:, :cents], "NaN" => missing) # replace "NaN" with missing
    df[:, :dollars] = replace(df[:, :dollars], missing => 0)  # replace missing with 0 
    df[:, :cents] = replace(df[:, :cents], missing => 0)  # replace missing with 0 
    df 
end 

function group_by_year(df) 
    gdf = @by(df, :year,
        :total_amt = sum(:dollars) + sum(:cents)  # calculate total amount of debt 
    )
    sort!(gdf) # sort by year 
    replace!(gdf.year, "missing" => "no year") # change missing values to string to allow for plotting
    return gdf 
end

function group_by_year_month(df)
    gdf = @by(df, [:year, :month],
        :total_amt = sum(:dollars) + sum(:cents)  # calculate total amount of debt 
    )
    
    return gdf 
end

#plot debt redeemed per year saved as svg 
function plot_debt(df::DataFrame, state, type)
    set_default_plot_size(40cm, 22.5cm)

    if type == "year"
        p_debt_date = Gadfly.plot(
            df,
            x=:year,
            y=:total_amt,
            Geom.bar,
            Guide.xlabel("Year"),
            Guide.ylabel("Total Debt (in dollars)"),
            Guide.title(uppercase(state)*" Debt Redeemed Per Year"),
            Gadfly.Theme(background_color = "white")
        )
        img = SVG("results/debt_per_year/year/" * uppercase(state) * "_debt_redeemed_per_year.svg", 40cm, 22.5cm) 
        draw(img, p_debt_date)  
    elseif type == "year_month"
        p_debt_date = Gadfly.plot(
            df,
            x=:year_month,
            y=:total_amt,
            Geom.bar,
            Guide.xlabel("Year"),
            Guide.ylabel("Total Debt (in dollars)"),
            Guide.title(uppercase(state)*" Debt Redeemed Per Year/Month"),
            Gadfly.Theme(background_color = "white")
        )

        img = SVG("results/debt_per_year/year_month/" * uppercase(state) * "_debt_redeemed_per_year_month.svg", 40cm, 22.5cm) 
        draw(img, p_debt_date)  
    end

    println(state)

end 

plot_debt (generic function with 1 method)

In [5]:
function group_post1795(state_df) 
    # filtering dataframes using multiple conditions 
    state_df_post1795 = @subset(state_df, :year_month .> DateTime(1795)) # filter out years before 1795
    state_df_post1795.total_amt = replace(state_df_post1795.total_amt, missing => 0) # replace missing values with 0
    state_df_sum = sum(state_df_post1795.total_amt) # sum total debt redeemed after 1795

    # remove rows where year is greater than 1795 
    state_df = @subset(state_df, :year_month .< DateTime(1795))
    
    # change datetime object to string 
    state_df.year_month = Dates.format.(state_df.year_month, "yyyy-mm")
    
    push!(state_df, [state_df_sum, "post-1795"], promote=true) # add total debt redeemed after 1795 to dataframe
    return state_df
end 

group_post1795 (generic function with 1 method)

In [6]:
function handle_missing_info(state_df)
    missing_info = @rsubset(state_df, ismissing(:year) || ismissing(:month)) # filter out rows with missing year or month\
    missing_info_sum = 0
    if nrow(missing_info) > 0
        missing_info_sum = sum(missing_info.dollars + missing_info.cents) # sum total debt redeemed with missing year or month
    end 
    return missing_info_sum
end 

handle_missing_info (generic function with 1 method)

In [7]:
# import cd_info 
cd_info = DataFrame(CSV.File("data/cd_info.csv"))
# store total amount and years of all states in a new dataframe 
all_states = DataFrame([[], []], [:year_month, :total_amt])

# loop through cd_info per state 
for i in 1:nrow(cd_info)
    state_df = DataFrame() # create new dataframe for each state
    state_row = cd_info[i, :]
    state_excel = XLSX.readxlsx(state_row[:file_path])
    state_sheet = state_excel["Sheet1"]
    
    # get year columns --> merge years columns into one column 
    year_col_indexes = split(state_row[:year_col], ",")
    for year_col_index in year_col_indexes
        year_col_range = year_col_index*string(state_row[:first_row])*":"*year_col_index*string(state_row[:last_row])
        year_col = state_sheet[year_col_range]
        
        if nrow(state_df) == 0 # if state_df is empty, create year column 
            state_df.year = vec(year_col)
        else 
            state_df.year .= coalesce(state_df.year, vec(year_col)) # append year col to state dataframe
        end 
    end
    
    # get month columns --> merge month columns into one column
    month_col_indexes = split(state_row[:month_col], ",")
    for month_col_index in month_col_indexes
        month_col_range = month_col_index*string(state_row[:first_row])*":"*month_col_index*string(state_row[:last_row])
        month_col = state_sheet[month_col_range]

        if !("month" in names(state_df)) 
            state_df.month = vec(month_col)
        else
            state_df.month .= coalesce(state_df.month, vec(month_col)) # append month col to state dataframe
        end 
    end
    
    # add dollar amount column to states dataframe 
    dollar_indexes = split(state_row[:dollars_col], ",") 
    for dollar_index in dollar_indexes
        dollar_col_range = dollar_index*string(state_row[:first_row])*":"*dollar_index*string(state_row[:last_row])
        dollar_col = state_sheet[dollar_col_range]
        state_df.dollars = vec(dollar_col)
    end

    # add cents amount column to states dataframe 
    if !ismissing(state_row[:cents_col]) # handle excel spreadsheets with no cents column 
        cents_indexes = split(state_row[:cents_col], ",")
        for cents_index in cents_indexes
            cents_col_range = cents_index*string(state_row[:first_row])*":"*cents_index*string(state_row[:last_row])
            cents_col = state_sheet[cents_col_range]
            state_df.cents = vec(cents_col) ./ 100 # convert cents to decimal
        end 
    else 
        state_df.cents = zeros(nrow(state_df)) # create column of zeros to allow for summing
    end

    state_df = clean_df(state_df) # clean dataframe to removeq missing values from :dollars and :cents columns
    missing_info_sum = handle_missing_info(state_df) # get total value of rows with missing info 
    dropmissing!(state_df, [:year, :month]) # drop rows with missing values in year, month columns

    state_df.year = string.(state_df.year)
    state_df.month = string.(state_df.month)

    # add state label 
    state_df.state = fill(state_row[:state], nrow(state_df))

    state_df_clean = clean_df(state_df) # clean table to remove missing values
    state_gdf = group_by_year_month(state_df_clean) # group by year and month and sum debt 

    # create new column that merges year and month 
    state_gdf.year_month = string.(state_gdf[:, :year], "-", state_gdf[:, :month])

    # convert year_month column to datetime object 
    state_gdf.year_month = DateTime.(state_gdf.year_month, DateFormat("yyyy-mm"))
    sort!(state_gdf, :year_month) # sort by year_month column   

    # remove year and month columns 
    select!(state_gdf, Not([:year, :month]))

    all_states = vcat(all_states, state_gdf) # append state dataframe to all_states dataframe

    state_gdf = group_post1795(state_gdf) # filter out years before 1795
    push!(state_gdf, [missing_info_sum, "missing info"], promote=true) # add total debt redeemed with missing info to dataframe

    plot_debt(state_gdf, state_row[:state], "year_month") # plot debt redeemed per year saved as svg
    #plot_debt(state_gdf, state_row[:state], "year") # plot debt redeemed per year saved as svg

    println(last(state_gdf, 5)) 
end 


ct


5×2 DataFrame
 Row │ total_amt      year_month   
     │ Float64        String       
─────┼─────────────────────────────
   1 │   467.11       1794-10
   2 │     1.23824e5  1794-11
   3 │     0.0        1794-12
   4 │   788.6        post-1795
   5 │ 13337.7        missing info

ga
5×2 DataFrame
 Row │ total_amt  year_month   
     │ Float64    String       
─────┼─────────────────────────
   1 │     89.53  1793-02
   2 │     27.29  1794-03
   3 │  23985.6   1794-11
   4 │      0.0   post-1795
   5 │      0.0   missing info


md
5×2 DataFrame
 Row │ total_amt    year_month   
     │ Float64      String       
─────┼───────────────────────────
   1 │   0.0        1794-10
   2 │  64.17       1794-11
   3 │ 130.0        1794-12
   4 │ 994.68       post-1795
   5 │   1.02597e5  missing info
nc
5×2 DataFrame
 Row │ total_amt  year_month   
     │ Float64    String       
─────┼─────────────────────────
   1 │   1005.93  1793-06
   2 │  12170.2   1794-02
   3 │   2231.66  1794-07
   4 │      0.0   post-1795
   5 │      0.0   missing info

nh
5×2 DataFrame
 Row │ total_amt  year_month   
     │ Float64    String       
─────┼─────────────────────────
   1 │   5104.4   1792-11
   2 │    658.99  1792-12
   3 │   4597.44  1793-01
   4 │      0.0   post-1795
   5 │      0.0   missing info
nj
5×2 DataFrame
 Row │ total_amt      year_month   
     │ Float64        String       
─────┼─────────────────────────────
   1 │  1006.2        1794-09
   2 │ 10160.9        1794-10
   3 │     3.74       1794-11
   4 │     1.73934e5  post-1795
   5 │   351.48       missing info


ny
5×2 DataFrame
 Row │ total_amt      year_month   
     │ Float64        String       
─────┼─────────────────────────────
   1 │ 40031.3        1794-09
   2 │ 47256.6        1794-10
   3 │  1495.89       1794-12
   4 │     1.70848e5  post-1795
   5 │  3351.63       missing info

pa
5×2 DataFrame
 Row │ total_amt  year_month   
     │ Float64    String       
─────┼─────────────────────────
   1 │    990.5   1792-12
   2 │    584.98  1793-01
   3 │  15698.8   1793-02
   4 │      0.0   post-1795
   5 │  76527.9   missing info

ri
5×2 DataFrame
 Row │ total_amt  year_month   
     │ Float64    String       
─────┼─────────────────────────
   1 │    213.94  1794-09
   2 │  59922.2   1794-11
   3 │      9.51  1794-12
   4 │      0.0   post-1795
   5 │  20673.4   missing info
sc
5×2 DataFrame
 Row │ total_amt      year_month   
     │ Float64        String       
─────┼─────────────────────────────
   1 │   363.98       1794-04
   2 │     2.41984e5  1794-10
   3 │  6417.45       1794-11
   4 │ 17033.8        post-1795
   5 │ 19148.7        missing info


va
5×2 DataFrame
 Row │ total_amt  year_month   
     │ Float64    String       
─────┼─────────────────────────
   1 │   1349.76  1794-08
   2 │    464.12  1794-11
   3 │    226.83  1794-12
   4 │   7691.96  post-1795
   5 │  40353.0   missing info


In [8]:
# plot united states 
all_states = @by(all_states, :year_month, :total_amt = sum(:total_amt)) # group by year and month and sum debt
all_states = all_states[!, [:total_amt, :year_month]] # swap columns
sort!(all_states, :year_month) # sort by year_month column
all_states = group_post1795(all_states) # filter out years before 1795

Row,total_amt,year_month
,Float64,String
1,19429.3,1790-01
2,0.0,1790-09
3,4.90071e5,1790-10
4,2.82428e5,1790-11
5,3.28619e5,1790-12
6,301538.0,1791-01
7,3.67543e5,1791-02
8,7.1012e5,1791-03
9,5.0777e5,1791-04


<h1>Pre-1790</h1>

In [9]:
# import pre-1790 debt data 
# get unique dates 
# group by year - sum debt 
# plot debt redeemed per year saved as svg 

pre1790 = DataFrame(CSV.File("../data/agg_debt_grouped.csv"))
pre1790.year = pre1790[:, "date of the certificate | year"]

# fix cents column in agg_debt_grouped.csv 
pre1790[:, "amount | 90th"] = getindex.(split.(pre1790[:, "amount | 90th"], "."), 1)
pre1790[:, "amount | 90th"] = replace.(pre1790[:, "amount | 90th"], "/" => "")
pre1790.cents = parse.(Float64, pre1790[:, "amount | 90th"]) ./ 100

pre1790.total_amt = pre1790[:, "amount | dollars"] + pre1790[:, "cents"]
pre1790.dollars = pre1790[:, "amount | dollars"]

pre1790.cents = ifelse.(pre1790.cents .>= 100, 0, pre1790.cents)

#clean 
pre1790_clean = clean_df(pre1790)

#group by year and sum 
pre1790_clean.year = coalesce.(pre1790_clean.year, 0)
pre1790_clean.year = Int.(pre1790_clean.year)
pre1790_clean.year = string.(pre1790_clean.year)
pre1790_gdf = group_by_year(pre1790_clean)
pre1790_gdf[1, "year"] = "no year"
sort!(pre1790_gdf)

Row,year,total_amt
,String,Float64
1,1776,51300.0
2,1777,4.9459e6
3,1778,8.1022e6
4,1779,2.81939e7
5,1780,1.4315e7
6,1781,2.1771e6
7,1783,1.04227e5
8,1784,1.06626e6
9,1785,1.53478e6


In [10]:
pre1790_gdf.total_amt = pre1790_gdf.total_amt ./ 1e6 # convert to millions

14-element Vector{Float64}:
  0.0513
  4.9459
  8.1022
 28.193929
 14.315
  2.1771
  0.10422725999999999
  1.06626137
  1.53478178
  1.022512745
  0.17197539
  0.06277835
  0.017804189999999998
 12.273124663332927

In [15]:
#plot 
p_bar = bar(pre1790_gdf.year, pre1790_gdf.total_amt, 
    title = "Debt Certificate Total Per Year",
    xlabel = "Year",
    ylabel = "Amount (in million dollars)",
    legend = false,
    background_color = "white",
    yaxis=[0, 30],
    top_margin=5mm,
)

# round to 2 decimal places
pre1790_gdf.total_amt = round.(pre1790_gdf.total_amt, digits=2)
annotate!(pre1790_gdf.year, pre1790_gdf.total_amt, pre1790_gdf.total_amt, annotationfontsizes=8, annotationvalign=:bottom)

savefig(p_bar, "results/debt_per_year/pre1790_debt_certificate_amts_per_year.svg")

"c:\\Users\\david\\OneDrive\\Documents\\SPEOC-pt-1\\cleaning_CD\\pre1790\\analysis\\results\\debt_per_year\\pre1790_debt_certificate_amts_per_year.svg"

In [16]:
#plot 
total_amt = sum(pre1790_gdf.total_amt)
pre1790_gdf.percent = pre1790_gdf.total_amt ./ total_amt

p_bar = bar(pre1790_gdf.year, pre1790_gdf.percent, 
    title = "Percent of Debt Certificate Total Per Year",
    xlabel = "Year",
    ylabel = "Amount (Percentage of Total)",
    legend = false,
    background_color = "white",
    yaxis=[0, 0.5],
    top_margin=5mm
)

# round to 2 decimal places
pre1790_gdf.percent = round.(pre1790_gdf.percent, digits=2)
annotate!(pre1790_gdf.year, pre1790_gdf.percent, pre1790_gdf.percent, annotationfontsizes=8, annotationvalign=:bottom)

savefig(p_bar, "results/debt_per_year/pre1790_debt_certificate_percent_per_year.svg")

"c:\\Users\\david\\OneDrive\\Documents\\SPEOC-pt-1\\cleaning_CD\\pre1790\\analysis\\results\\debt_per_year\\pre1790_debt_certificate_percent_per_year.svg"